In [63]:
# Import Dependencies
import pandas as pd
import os

In [71]:
# Import CSV and convert to dataframes
test_df = pd.read_csv(os.path.join("Resources", "school_test.csv"))

#### Clean test dataframe

In [72]:
# Drop and rename columns
test_df = test_df[["DistrictCode", "SchoolCode", "Test", "Subject", "School_Avg", "State_avg"]]
test_df = test_df.rename(columns={"DistrictCode": "DISTRICT_CODE", "SchoolCode": "SCHOOL_CODE",\
                                  "SchoolCode": "SCHOOL_CODE", "Test": "TEST","School_Avg": "SCH_AVG", \
                                  "State_avg": "STATE_AVG"})

# Verify no missing data
test_df.isnull().sum()

# Drop any duplicate rows
test_df.drop_duplicates()

# Add leading zeros to district and school codes
test_df["DISTRICT_CODE"] = test_df["DISTRICT_CODE"].apply(lambda x: "{0:0>4}".format(x))
test_df["SCHOOL_CODE"] = test_df["SCHOOL_CODE"].apply(lambda x: "{0:0>3}".format(x))


# Create unique key column from district and school codes
test_df["DS_CODE"] = test_df["DISTRICT_CODE"].map(str) + "-" + test_df["SCHOOL_CODE"].map(str)

# Review ACT scores to verify no missing values and scores within valid range
ACT_df = test_df[test_df['TEST'].str.contains('ACT')]
ACT_df.SCH_AVG.unique()
ACT_df.STATE_AVG.unique()

# Review SAT scores to verify no missing values and scores within valid range
SAT_df = test_df[test_df['TEST'].str.contains('SAT')]
SAT_df.SCH_AVG.unique()
SAT_df.STATE_AVG.unique()

# Replace missing values with None and cast as integer
test_df = test_df.replace(["N", "*"], None)
test_df["SCH_AVG"] = test_df["SCH_AVG"].astype('int64')

# Verify values are of the correct type
test_df.dtypes

# Filter only SAT scores and separate into columns for Math and English
test_df = test_df.loc[test_df['TEST'] == "SAT"]
math_df = test_df.loc[test_df["Subject"] == "Math"]
english_df = test_df.loc[test_df["Subject"] == "Reading and Writing"]
english_df = english_df[["DS_CODE", "Subject", "SCH_AVG", "STATE_AVG"]]
english_df = english_df.rename(columns={"SCH_AVG": "ENG_SCH_AVG", "STATE_AVG": "ENG_STATE_AVG"})
math_df = math_df.rename(columns={"SCH_AVG": "MATH_SCH_AVG", "STATE_AVG": "MATH_STATE_AVG"})
math_df = math_df[["Subject", "MATH_SCH_AVG", "MATH_STATE_AVG", "DS_CODE"]]
test_df = math_df.merge(english_df, on="DS_CODE", how="outer")
test_df = test_df.drop(labels = {"Subject_x", "Subject_y"}, axis = 1)

# Verify no missing data
test_df.isnull().sum()

DISTRICT_CODE    object
SCHOOL_CODE      object
TEST             object
Subject          object
SCH_AVG           int64
STATE_AVG         int64
DS_CODE          object
dtype: object